#Import necessary library

In [1]:
!pip install transformers
!pip install Sastrawi

import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import f1_score
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 17.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 5.5 MB/s eta 0:00:00


#Load & Preprocess the data

In [2]:
df = pd.read_csv("clean_tweets.csv")
df.dropna(inplace=True)

# Remove stop words using Sastrawi library
factory = StopWordRemoverFactory()
stop_words = factory.create_stop_word_remover()
df['Tweet'] = df['Tweet'].apply(lambda x: stop_words.remove(x))

# Map labels to integers
label_map = {"positive": 1, "negative": 0}
df['label'] = df['label'].map(label_map)

# Split data into train and test sets
train_text, test_text, train_labels, test_labels = train_test_split(df['Tweet'].values, df['label'].values, test_size=0.2, random_state=42)


#Tokenize data using BERT tokenizer

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)

# Encode train and test data using BERT tokenizer
train_encodings = tokenizer(train_text.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_text.tolist(), truncation=True, padding=True)

# Convert encoded inputs to PyTorch tensors
train_inputs = torch.tensor(train_encodings['input_ids'])
train_masks = torch.tensor(train_encodings['attention_mask'])
train_labels = torch.tensor(train_labels)
test_inputs = torch.tensor(test_encodings['input_ids'])
test_masks = torch.tensor(test_encodings['attention_mask'])
test_labels = torch.tensor(test_labels)

# Create data loaders for efficient batching
batch_size = 16
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


#Train the model

In [5]:
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2, output_attentions=False, output_hidden_states=False)

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set up optimizer and scheduler
epochs = 5
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

from tqdm import tqdm

# Train the model
for epoch in range(epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch+1}', leave=False)
    for step, batch in enumerate(progress_bar):
        batch_inputs = batch[0].to(device)
        batch_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)
        optimizer.zero_grad()
        outputs = model(batch_inputs, attention_mask=batch_masks, labels=batch_labels)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        avg_train_loss = total_loss / (step+1)
        progress_bar.set_postfix({'training_loss': avg_train_loss})
    print(f'Epoch: {epoch + 1}, Training Loss: {avg_train_loss}')




Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Epoch: 1, Training Loss: 0.4742465856209607


Epoch: 2, Training Loss: 0.29414802670583756


Epoch: 3, Training Loss: 0.21983868898495926


Epoch: 4, Training Loss: 0.1722782380564112


Epoch: 5, Training Loss: 0.12001829272575519


#Save the trained model

In [6]:
model_path = 'bert_sa_sentiment_model.pt'
torch.save(model.state_dict(), model_path)

#Evaluate the model

In [11]:
# Load the saved model
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2, output_attentions=False, output_hidden_states=False)
model.load_state_dict(torch.load(model_path))
model.to(device)

# Evaluate the model on test data
model.eval()
total_preds = []
total_labels = []
with torch.no_grad():
    for batch in test_dataloader:
        batch_inputs = batch[0].to(device)
        batch_masks = batch[1].to(device)
        batch_labels = batch[2].to('cpu')
        outputs = model(batch_inputs, attention_mask=batch_masks)
        logits = outputs[0]
        preds = torch.argmax(logits, axis=1)
        total_preds.extend(preds)
        total_labels.extend(batch_labels)

total_labels = torch.tensor(total_labels)
total_preds = torch.tensor(total_preds)

# Calculate F1 score
f1 = f1_score(total_labels.detach().cpu().numpy(), total_preds.detach().cpu().numpy(), average='weighted')
print(f'Weighted F1 score: {f1:.4f}')


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Weighted F1 score: 0.8599


#Use the trained model for inference

In [12]:
def predict_sentiment(text):
    # Remove stop words from input text
    text = stop_words.remove(text)
    # Tokenize input text using BERT tokenizer
    encoding = tokenizer.encode_plus(text, max_length=64, truncation=True, padding='max_length', add_special_tokens=True, return_attention_mask=True, return_tensors='pt')
    inputs = encoding['input_ids'].to(device)
    masks = encoding['attention_mask'].to(device)
    # Pass input through the trained model and return predicted sentiment
    with torch.no_grad():
        outputs = model(inputs, attention_mask=masks)
        logits = outputs[0]
        probs = torch.softmax(logits, dim=1)
        _, pred_label = torch.max(probs, dim=1)
    return 'positive' if pred_label == 1 else 'negative'


#Result

In [17]:
text = "Bodoh kamu"
sentiment = predict_sentiment(text)
print(sentiment)


negative


In [18]:
text = "Kamu sangat pintar"
sentiment = predict_sentiment(text)
print(sentiment)

positive


In [19]:
text = "Dasar ga jelas lo anjing"
sentiment = predict_sentiment(text)
print(sentiment)

negative


In [20]:
text = "Wah pemandangan yang sangat indah"
sentiment = predict_sentiment(text)
print(sentiment)

positive
